# Part I. ETL Pipeline for Pre-Processing the Files

## CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra portion of project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries

## Create queries to ask the following three questions of the data

### 1. Get artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1 : Get artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
# Create Table for Query 1
create_table_1 = "create table if not exists song_play_history (sessionId int, itemInSession int, artist text, length float, song text, PRIMARY KEY(sessionId,itemInSession))"
try:
    session.execute(create_table_1)
except Exception as e:
    print(e)                   

In [10]:
# Insert data in song_play_history
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERTING statements into the `insert1` variable
        insert_1 = "insert into song_play_history (sessionId, itemInSession, artist, length, song)"
        insert_1 = insert_1 + "values (%s,%s,%s,%s,%s)"
        ## Insert appropriate data in respective columns
        session.execute(insert_1, (int(line[8]),int(line[3]), line[0], float(line[5]),line[9]))

In [11]:
# SELECT to verify results
select_1="select artist, song, length from song_play_history where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(select_1)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2 : Get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
# Create Table for Query 2
create_table_2 = "create table if not exists song_playlist_session (sessionId int, userId int, artist text, itemInSession int, song text, firstName text, lastName text, PRIMARY KEY((sessionId,userId),itemInSession))"

try:
    session.execute(create_table_2)
except Exception as e:
    print(e)

In [13]:
# Insert data in song_playlist_session table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_2 = "insert into song_playlist_session (sessionid, userid, artist, itemInSession, song, firstName, lastName)"
        insert_2 = insert_2 + "values (%s,%s,%s,%s,%s,%s,%s)"
        # Insert appropriate data in respective columns
        session.execute(insert_2, (int(line[8]), int(line[10]), line[0], int(line[3]), (line[9]), (line[1]), line[4]))

In [14]:
# Select to verify results
select_2="select artist, song, firstName, lastName from song_playlist_session where userid = 10 and sessionid = 182"
try:
    rows = session.execute(select_2)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3 :  Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [15]:
## Create table for query3
create_table_3 = "create table if not exists songplay_username (userId int, song text, firstName text, lastName text, PRIMARY KEY(userId,song, firstName))"

try:
    session.execute(create_table_3)
except Exception as e:
    print(e)                    

In [16]:
# Insert dat into songplay_username
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT appropriate data into respective columns
        insert_3 = "insert into songplay_username (userId, song, firstName, lastName)"
        insert_3 = insert_3 + "values (%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_3, (int(line[10]),line[9], line[1],line[4]))

In [17]:
select_3="select firstName, lastName from songplay_username where song='All Hands Against His Own'"
try:
    rows = session.execute(select_3)
except Exception as e:
    print(e)
for row in rows:
    print (row.firstname, row.lastname)
                    

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Droping tables

In [18]:
session.execute("drop table song_play_history")
session.execute("drop table song_playlist_session")
session.execute("drop table songplay_username")

### Closing connection¶

In [20]:
session.shutdown()
cluster.shutdown()